In [1]:
import requests
from bs4 import BeautifulSoup
from persiantools.jdatetime import JalaliDate
from IPython.display import display

In [2]:
# convert curl to python
# https://curlconverter.com/

In [3]:
import requests

cookies = {
    'Province': '8',
    'City': '95',
    'ASP.NET_SessionId': 'teufzlt0gijv0wawu1jyqy4k',
    '__AntiXsrfToken': '1977b87c4a7244198ad779bf7a74406a',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:126.0) Gecko/20100101 Firefox/126.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.time.ir',
    'DNT': '1',
    'Sec-GPC': '1',
    'Connection': 'keep-alive',
    'Referer': 'https://www.time.ir/fa/eventyear-%d8%aa%d9%82%d9%88%db%8c%d9%85-%d8%b3%d8%a7%d9%84%db%8c%d8%a7%d9%86%d9%87',
    # 'Cookie': 'Province=8; City=95; ASP.NET_SessionId=teufzlt0gijv0wawu1jyqy4k; __AntiXsrfToken=1977b87c4a7244198ad779bf7a74406a',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Priority': 'u=1',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

data = {
    'ctl00_rssmStyleSheet_TSSM': '',
    'ctl00_rsmAll_TSM': '',
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__VIEWSTATE': 'VmTFKMx5W7vlHF/XJHm6cnwLTE+TkQh0zkT9nOOFk6DcczP2Tga5FQDdlkbL96p4g5TZ7szT5gCFfUPtGcwhIgXWuUtyFZn9N+lhggAz2uZctG74',
    'ctl00$cphTop$Sampa_Web_View_EventUI_EventYearCalendar10cphTop_3417$txtYear': '1402',
    'ctl00$cphTop$Sampa_Web_View_EventUI_EventYearCalendar10cphTop_3417$btnGo': 'برو به سال',
    '__VIEWSTATEGENERATOR': '65E7F3AF',
}

response = requests.post(
    'https://www.time.ir/fa/eventyear-%D8%AA%D9%82%D9%88%DB%8C%D9%85-%D8%B3%D8%A7%D9%84%DB%8C%D8%A7%D9%86%D9%87',
    cookies=cookies,
    headers=headers,
    data=data,
)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html itemscope="" itemtype="http://schema.org/WebPage" lang="fa" xml:lang="fa">
<head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="LF3TA" name="fontiran.com:license"/><meta content="UECNB" name="fontiran.com:license"/><meta content="PN88C" name="fontiran.com:license"/><title>
	تقویم سالیانه | تایم
</title>
<meta content="index, follow" name="robots"/>
<link href="/App_Themes/favicon.ico" rel="shortcut icon" type="image/x-icon"/><meta content="ساعت و تقویم ایران" name="apple-mobile-web-app-title"/><link href="/App_Themes/touch-icon-iphone.png" rel="apple-touch-icon"/><link href="/App_Themes/touch-icon-ipad.png" rel="apple-touch-icon" sizes="76x76"/><link href="/App_Themes/touch-icon-iphone-retina.png" rel="apple-touch-icon" sizes="120x120"/><link href="/App_Themes/touch-icon-ipad-retina.png" rel="apple-touch-icon" sizes="152x152"/>
<meta content="https:/

In [5]:
from holiday import Holiday
import pandas as pd
h = Holiday(soup)
holidays = h.get_holidays(add_noroze=True, add_thursday=True)
holidays[:5]

[JalaliDate(1403, 1, 1, Chaharshanbeh),
 JalaliDate(1403, 1, 2, Panjshanbeh),
 JalaliDate(1403, 1, 3, Jomeh),
 JalaliDate(1403, 1, 4, Shanbeh),
 JalaliDate(1403, 1, 10, Jomeh)]

In [6]:
def calculate_holidays_distance(day:JalaliDate ,holidays: list[JalaliDate]) -> list[int]:
    return list(map(lambda h: (h - day).days, holidays))
calculate_holidays_distance(JalaliDate(1402,6,1), holidays)[:5]

[210, 211, 212, 213, 219]

In [7]:
s = pd.Series(calculate_holidays_distance(JalaliDate(1402,6,2), holidays))
print(s[s > 0].nsmallest(5).tolist())
print(s[s <= 0].nlargest(5).tolist())
print([-i for i in s[s <= 0].nlargest(5)])

[209, 210, 211, 212, 213]
[]
[]


In [8]:
import datetime
def get_n_next_holiday(n:int, day:JalaliDate, holidays: list[JalaliDate]) -> list[int]:
    s = pd.Series(calculate_holidays_distance(day, holidays))
    return s[s > 0].nsmallest(n).tolist()
def get_n_previous_holiday(n:int, day:JalaliDate, holidays: list[JalaliDate], abs_return:bool=True) -> list[int]:
    s = pd.Series(calculate_holidays_distance(day, holidays))
    if abs_return:
        return [-i for i in s[s <= 0].nlargest(n)]
    return s[s <= 0].nlargest(n).tolist()
def get_holidays_this_week(day:JalaliDate, holidays: list[JalaliDate]) -> int:
    saturday = day - datetime.timedelta(day.weekday())
    s = pd.Series(calculate_holidays_distance(saturday, holidays))
    return s[(7 > s) & (s >= 0)].count()

d = JalaliDate(1402,6,12)
print(get_n_next_holiday(5, d, holidays))
print(get_n_previous_holiday(5, d, holidays))
print(get_holidays_this_week(d, holidays))

[199, 200, 201, 202, 203]
[]
0


In [9]:
from holiday import HolidayCalculator
hc = HolidayCalculator(holidays, d)
print(hc.get_holidays_distance())
print(hc.get_n_next_holiday(5))
print(hc.get_n_previous_holiday(5))
print(hc.get_holidays_this_week())
print(hc.get_holidays_this_week(True))

TypeError: 'function' object is not iterable

In [ ]:
h_map = dict()
n = 5
for day in result['DateReserve'].unique():
    calculator = HolidayCalculator(holidays, day, have_weekend=False)
    h_map[day] = {
        'HolidayInWeekCount': calculator.get_holidays_this_week(True),
        'HolidayInWeek': calculator.get_holidays_this_week(),
        'NextHoliday': calculator.get_n_next_holiday(n),
        'PreviousHoliday': calculator.get_n_previous_holiday(n)
    }

In [ ]:
# with open("date_140x.html", "w") as text_file:
#     text_file.write(response.text)

In [ ]:
holidays = []
for i in [1401,1402,1403]:
    with open(f"date/date_{i}.html", 'r') as file:
        file_content = file.read()
        soup = BeautifulSoup(file_content, 'html.parser')
        h = Holiday(soup)
        holidays_year = h.get_holidays(add_noroze=True, add_thursday=True)
        holidays.extend(holidays_year)
holidays

[JalaliDate(1401, 1, 1, Doshanbeh),
 JalaliDate(1401, 1, 2, Seshanbeh),
 JalaliDate(1401, 1, 3, Chaharshanbeh),
 JalaliDate(1401, 1, 4, Panjshanbeh),
 JalaliDate(1401, 1, 5, Jomeh),
 JalaliDate(1401, 1, 12, Jomeh),
 JalaliDate(1401, 1, 13, Shanbeh),
 JalaliDate(1401, 1, 19, Jomeh),
 JalaliDate(1401, 1, 26, Jomeh),
 JalaliDate(1401, 2, 2, Jomeh),
 JalaliDate(1401, 2, 3, Shanbeh),
 JalaliDate(1401, 2, 9, Jomeh),
 JalaliDate(1401, 2, 13, Seshanbeh),
 JalaliDate(1401, 2, 14, Chaharshanbeh),
 JalaliDate(1401, 2, 16, Jomeh),
 JalaliDate(1401, 2, 23, Jomeh),
 JalaliDate(1401, 2, 30, Jomeh),
 JalaliDate(1401, 3, 6, Jomeh),
 JalaliDate(1401, 3, 13, Jomeh),
 JalaliDate(1401, 3, 14, Shanbeh),
 JalaliDate(1401, 3, 15, Yekshanbeh),
 JalaliDate(1401, 3, 20, Jomeh),
 JalaliDate(1401, 3, 27, Jomeh),
 JalaliDate(1401, 4, 3, Jomeh),
 JalaliDate(1401, 4, 10, Jomeh),
 JalaliDate(1401, 4, 17, Jomeh),
 JalaliDate(1401, 4, 19, Yekshanbeh),
 JalaliDate(1401, 4, 24, Jomeh),
 JalaliDate(1401, 4, 27, Doshanbeh),